<a target="_blank" href="https://colab.research.google.com/github/WSU-CS1410-AA/cs1410-notebooks/blob/main/Notebook09-operator_overloading.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Operator Overloading

Early on in this course, we learned about the basic (primitive) data types that C++ supports such as `int`, `short`, `long`, `double`, and `float`. These data types support  certain operators such as `+`, `-`, `*`, and `/`, and `%` which allow us to do something like this:

```cpp
cout << 6 + 7   << endl   // 13
     << 6 - 5   << endl   // 1
     << 4 * 3   << endl   // 12
     << 7 / 2   << endl   // 3
     << 7 % 2   << endl   // 1
     << 7 / 2.0 << endl;  // 3.5
```

These basic types also support:
* relational operators such as `<`, `>`, `<=`, `>=`, `==`, and `!=`:

```cpp
cout << (6 < 7) << endl         // 1
     << (7 > 5) << endl         // 1
     << (4 >= 3) << endl        // 1
     << (2 <= 7) << endl        // 1
     << (9 == 9) << endl        // 1
     << (7.1 != 2.0) << endl;   // 1
```

* logical operators such as `&&` (AND), `||` (OR), and `!` (NOT):
```cpp
int x = 4;
cout << (x < 7 || x > 5) << endl;
cout << (x != 5 && x != 7) << endl;

```

* assignment operators:

```cpp
int ans = 27;

ans += 10;
cout << ans << ", ";
ans -= 7;
cout << ans << ", ";
ans *= 2;
cout << ans << ", ";
ans /= 3;
cout << ans << ", ";
ans %= 3;
cout << ans << endl;
```

* increment/decrement `--`/`++` operators:

```cpp
int count = 10;

cout << "count=" << count++ << endl;  // count=10
cout << "count=" << --count << endl;  // count=10
cout << "count=" << count-- << endl;  // count=10
cout << "count=" << count << endl;    // count=9
```

* output/input `<<`/`>>` operators:

```cpp
double num;
cout << "Enter a number:" << endl;
cin >> num;
cout << num;
```

* indexing operator `[]` for arrays of these types:

```cpp
char vowels[] = { 'a', 'e', 'i', 'o', 'u' };
cout << vowels[0] << endl;
cout << vowels[4] << endl;
```

The great thing about these operators is that they make using integers and decimals easy and efficient. The question is can we use these operators with objects of our own classes. For example, given two distance objects:

```c++
Distance d1, d2;
```
can we do:

```c++
Distance d3 = d1 + d2;
```
instead of:

```c++
Distance d3 = Distance.add(d1, d2);
```
The answer is: YES, we can, BUT we have to tell the C++ compiler what these operators (`+` in this example) do to objects of our own classes. C++ calls this **operator overloading**, and being able to overload operators sets C++ apart from many languages and gives a lot of power and flexibility.

Overloading operators involves thinking of them as functions. For example, having two objects `a` and `b` we can think of the expression `a + b` as `a.operator+(b)`. Similarly the unary operator `++a` can be thought of as `a.operator++()`. Here `operator+` and `operator++` are treated like member class functions.

We can also think of the expression `cout << a` as `operator<<(cout, a)`. Similarly `cout >> b` can be thought of as `operator<<(cin, b)`. These last two expressions are different in the sense that they involve not only our own objects (`a` and `b`) but also `cin` and `cout` which we are using for input/output but are not owers to change. Hence `operator<<` and `operator>>` are treated as simple non-class functions (that, as we will see later, are **friendly** to the classes of objects `a` and `b`).

### CODING CHALLENGE 1
Within the multi-line comment in the code cell below, think of the following operators as functions and write down what these functions would like. See the first example.

```c++
t || f
--count
!b
a += 3
n <= m
cin >> x
cout << y
```

In [7]:
%%writefile ch01.cpp

/* TODO

t.operator||(f)
count.operator--()
b.operator!()
a.operator+=(3)
n.operator<=(m)
cin.operator>>(x)
cout.operator<<(y)

*/

Writing ch01.cpp


## Two ways to overload operators

C++ gives us two ways to overload operators in our classes:
* using member functions
* Using friend functions. These are regular functions marked as *friends* to give them access to the private and protected members of the class.

Let us go through an example, first. Here we have a class like this:

In [8]:
%%writefile ex01.cpp

#include <iostream>

using namespace std;

// Version 1
class Counter {
private:
     int count;
     int initial;
protected:
     int step;
public:
    Counter(int i): initial(i), count(i), step(1){}
    int next(){ return count += step; }
    int prev(){ return count -= step; }
    int current(){ return count; }
    void reset(){ count = initial; }
};

Writing ex01.cpp


And we think it makes sense to use the operator ++ to mean `next()` and -- to mean `prev()` and would like the following code to work. Right now it does not. Uncomment the following code cell to see what kind of errors you are getting.

In [9]:
%%writefile -a ex01.cpp

int main(){
  /*
  Counter c(0);
  ++c; // equivalent ot calling c.next()
  cout << c; // Prints the current value of the counter
  --c; // equivalent to calling c.prev()
  cout << c;
  */

  return 0;
}

Appending to ex01.cpp


In [ ]:
!g++ -std=c++17 ex01.cpp -o ex01
!./ex01

To make operators `++`, `--`, and `<<` work for counter objects, the `Counter` class must overload these operators

From the above coding challenge, we can think of these operators as functions like this:
* `++c` is equivalent to `c.operator++()`
* `--c` is equivalent to `c.operator--()`
* `cout << c` is equivalent to `operator++(cout, c)`

Now we look at the leftmost operands for these operators. For both `++` and `--`, `c`,  which is a `Counter` object, is the leftmost operand. For the `<<` operand,  `cout` is leftmost operand.

Because the object `c` belongs to our own `Counter` class and we can easily add functions to it, we use member functions to overload the `++` and `--` operators.

On the other side, `cout` is an output stream object of a class given to us by the C++ standard library to which we cannot add member functions. Therefore we use a friend function instead to overload the `<<` operator.

In C++, all operator-overloading functions (member or friend) must have a name that starts with the keyword `operator` followed by the operator symbol(s).

Here is the `Counter` class after overloading the `++`, `--`, and `<<` operators.

In [10]:
%%writefile ex02.cpp

#include <iostream>

using namespace std;

// Version 2
class Counter {
private:
     int count;
     int initial;
protected:
     int step;
public:
    Counter(int i): initial(i), count(i), step(1){}
    int next(){ return count += step; }
    int prev(){ return count -= step; }
    int current(){ return count; }
    void reset(){ count = initial; }

    int operator++(){ return next(); } // Overloads prefix ++
    int operator--(){ return prev(); } // Overloads prefix --

    friend ostream& operator<<(ostream& out, Counter& c){ // Overloads <<
        out << c.count << endl;
        return out;
    }
};

Writing ex02.cpp


Having these operator functions tells the C++ compiler what to do when it sees these the `++`, `--`, and `<<` operators used for objects of the `Counter` class. In other words, we can now run the following code without errors:

In [11]:
%%writefile -a ex02.cpp

int main(){
  Counter c(0);
  ++c; // equivalent ot calling c.next()
  cout << c; // Prints the current value of the counter
  --c; // equivalent to calling c.prev()
  cout << c;

  return 0;
}

Appending to ex02.cpp


In [12]:
!g++ -std=c++17 ex02.cpp -o ex02
!./ex02

1
0


We must keep in mind that a **friend function** is not a member function even if the friend function is defined inside the class. It's just a regular function designated as a friend. Being a friend allows the function to access the private and protected members of this class's objects. In the `Counter` class, the `operator<<` function, being a friend of the `Counter` class, is able to access the private member `count` of the `c` object of the `Counter` class.

To summarize, when overloading an operator we need to think of it as a function and find out its leftmost operand:
* If the leftmost operand is an object of the class we are overloading, then use a member function
* If the leftmost operand is not an object of the class we are overloading, then use a friend function.

Pay attention to the signature of the friend `operator<<` function. It's a pattern that is repeated every time you want your object to work with `cout`. There is a similar pattern for `operator>>` which is used to make your object work with `cin`.

Let us create another counter class by overloading four more operators:
* the `+` operator such that `c + 5` means add 5 to c's count.
* the `-` operator such that `c - 6` means subtract 6 from c's count.
* the `==` operator such that `c1 == c2` returns true if `c1` has the same values for count, step, and initial as those of `c2`. It returns false otherwise.
* the `!=` operator which is the opposite of `==`.
* the `>>` operator such that `cin >> c` prompts the user to enter three integers: one for initial, another for count, and the last for step.

Here is the new  counter class.

In [13]:
%%writefile ex03.cpp

#include <iostream>

using namespace std;

// Version 3
class Counter {
private:
     int count;
     int initial;
protected:
     int step;
public:
    Counter(int i): initial(i), count(i), step(1){}
    int next(){ return count += step; }
    int prev(){ return count -= step; }
    int current(){ return count; }
    void reset(){ count = initial; }

    int operator++(){ return next(); } // Overloads prefix ++
    int operator--(){ return prev(); } // Overloads prefix --

    friend ostream& operator<<(ostream& out, Counter& c){ // Overloads <<
        out << c.count;
        return out;
    }

    // The new five operators
    void operator+(int n){ count += n; }
    void operator-(int n){ count -= n; }
    bool operator==(Counter& c){
        return this->initial == c.initial && this->count == c.count && this->step == c.step;
    }
    bool operator!=(Counter& c) { return !(*this == c); }
    friend istream& operator>>(istream& in, Counter& c){
        cout << "Enter counter initial, count, and step: ";
        in >> c.initial >>  c.count >> c.step;
        return in;
    }
};

Writing ex03.cpp


which allows us to run something like this:

In [14]:
%%writefile -a ex03.cpp

int main(){
  Counter c3(0);
  cin >> c3;
  cout << c3 << endl;
  ++c3;
  cout << c3 << endl;
  c3 + 9;
  cout << c3 << endl;
  --c3;
  cout << c3 << endl;
  c3 - 2;
  cout << (c3 == c3) << endl;
  cout << (c3 != c3) << endl;

  return 0;
}

Appending to ex03.cpp


In [15]:
!g++ -std=c++17 ex03.cpp -o ex03
!./ex03

Enter counter initial, count, and step: ^C


## CODING CHALLENGE 2

Refactor (modify) the class below so as to overload (implement) the following operators:
* the `>>` operator such that `cin >> d` prompts the user to enter two integers: feet and inches.
* the `<<` operator such that `cout << d` prints out the distance `d` in a format like this: `12' 3"`.
* the `+` operator such that `d + 5` means add 5 feed to d.
* the `--` operator such that `--d` means subtract d's fee by 1 foot.
* the `==` operator such that `d1 == d2` returns true if `d1` has the same feet and inches as `d2`.

In the `main` function , create two distance objects `d1` and `d2` and use them to call and test all the five overloaded operators.

In [18]:
%%writefile ch02.cpp

#include <iostream>
using namespace std;

class Distance{

private:
    int feet;
    int inches;

public:
    Distance(): feet(0), inches(0){}
    Distance(int feet, int inches): feet(feet), inches(inches){}
    Distance& add(const Distance& d){
        this->feet += d.feet + (this->inches + d.inches) / 12;
        this->inches = (this->inches + d.inches) % 12;
        return *this;
    }
    Distance& add(int feet){
        this->feet += feet;
        return *this;
    }
    Distance& add(int feet, int inches){
        this->feet += feet + (this->inches + inches) / 12;
        this->inches = (this->inches + inches) % 12;
        return *this;
    }

    Distance& show(){
        cout << this->feet << "' " << this->inches << "\"" << endl;
        return *this;
    }
    void operator+(int feet){ this->feet += feet; }
    int operator--(){ return this->feet -= 1; }
    bool operator==(Distance& d) { return this->feet == d.feet && this->inches == d.inches; }
    friend ostream& operator<<(ostream& out, Distance& d){
        out << d.feet << "' " << d.inches << "\"";
        return out;
    }
    friend istream& operator>>(istream& in, Distance& d){
        cout << "Enter feet and inches (integers):" << endl;
        in >> d.feet >> d.inches;
        return in;
    }
};

int main(){
  // TODO
  Distance d1(0, 0);
  Distance d2(0, 0);
  cin >> d1;
  cin >> d2;
  cout << d1 << endl;
  cout << d2 << endl;
  d1 + 6;
  cout << d1 << endl;
  --d2;
  cout << d2 << endl;
  cout << (d1 == d2) << endl;

  return 0;
}

Overwriting ch02.cpp


In [17]:
!g++ -std=c++17 ch02.cpp -o ch02
!./ch02

Enter feet and inches (integers):
5 9
Enter feet and inches (integers):
6 2
5' 9"
6' 2"
11' 9"
5' 2"
0


## Another look at the counter class

Right now the latest version of the `Counter` class suppors something like this:

```cpp
Counter c4(0);
++c4;
--c4;
c4 + 9;
c4 - 2;
```

However, it does not support something like this.

```cpp
c4++;
c4--;
9 + c4;
2 - c4;
```

How do we support the postfix operations `c4++` and `c4--` and what makes them different from the already implemented  prefix operators `++c4` and `--c4`? It turns out that to distinguish between prefix and postfix operators, C++ uses a different signature for the postfix operators from the prefix operators. While the prefix operators use the following definitions:

```c++
int operator++(){ return next(); }
int operator--(){ return prev(); }
```
the postfix operators use the following definitions:

```c++
int operator++(int){ return next(); }
int operator--(int){ return prev(); }
```

The `int` parameter is a dummy unused parameter meant to only tell C++ which definition is for prefix and which is postfix operators.

On the other hand, the expressions `9 + c4` and `2 - c4` both have integer leftmost operands. That means we need to use a friend function for these.

Here is a counter class that supports these new operators.

In [19]:
%%writefile ex04.cpp

#include <iostream>

using namespace std;

// Version 4
class Counter {
private:
     int count;
     int initial;
protected:
     int step;
public:
    Counter(int i): initial(i), count(i), step(1){}
    int next(){ return count += step; }
    int prev(){ return count -= step; }
    int current(){ return count; }
    void reset(){ count = initial; }

    int operator++(){ return next(); } // Overloads prefix ++
    int operator--(){ return prev(); } // Overloads prefix --

    friend ostream& operator<<(ostream& out, Counter& c){ // Overloads <<
        out << c.count;
        return out;
    }

    // The new five operators
    void operator+(int n){ count += n; }
    void operator-(int n){ count -= n; }
    bool operator==(Counter& c){
        return this->initial == c.initial && this->count == c.count && this->step == c.step;
    }
    bool operator!=(Counter& c) { return !(*this == c); }
    friend istream& operator>>(istream& in, Counter& c){
        cout << "Enter counter initial, count, and step: ";
        in >> c.initial >>  c.count >> c.step;
        return in;
    }

    int operator++(int){ return next(); } // Overloads postfix ++
    int operator--(int){ return prev(); } // Overloads postfix --

    friend void operator+(int n, Counter& c){ c.count += n; }
    friend void operator-(int n, Counter& c){ c.count -= n; }

};

Writing ex04.cpp


Let's test these newly overloaded operators.

In [20]:
%%writefile -a ex04.cpp

int main(){
  Counter cc(0);
  cc++;
  cc--;
  9 + cc;
  2 - cc;
  cout << cc;

  return 0;
}

Appending to ex04.cpp


In [21]:
!g++ -std=c++17 ex04.cpp -o ex04
!./ex04

7

## Overloading `[]`

Finally here is an example of overloading the `[]` operator which is typically used for indexing arrays.

In [22]:
%%writefile ex05.cpp

#include <iostream>

using namespace std;

class Complex {
private:
    double real, imaginary;

public:
    Complex(double r = 0, double i= 0): real(r), imaginary(i){}

    double operator[](int index) const { //READ OONLY: real value at index 0, imaginary at any other index;
        switch(index){
        case 0: return real;
        default: return imaginary;
        }
    }

    double& operator[](int index){ //WRITE: real value at index 0, imaginary at any other index;
        switch(index){
        case 0: return real;
        default: return imaginary;
        }
    }
};

Writing ex05.cpp


In [23]:
%%writefile -a ex05.cpp

int main(){
  Complex xx(5,6);

  cout << xx[0] << endl;
  xx[1] = 9;
  cout << xx[1];

  return 0;
}

Appending to ex05.cpp


In [24]:
!g++ -std=c++17 ex05.cpp -o ex05
!./ex05

5
9

## Operator overloading restrictions
As powerful as operator overloading can be, it comes with a few restrictions:  
* You cannot introduce a new operator that C++ does not already support.
* You cannot alter the precedence of the operator you are overloading.
* You cannot change the number of operands that an operator takes.
* Overloaded operators cannot have default arguments.
* The following operators cannot be overloaded: `.`, `::`, `.*`, `?:`, and `sizeof`

## Another detailed operator overloading example

In this example, we have a class named `MyIntegerList` representing a list of zero or more integers much like an array or a vector. This class has member functions for:
* Finding how many integers are in the list
* Returning the first integer in the list
* Returning the last integer in the list
* Returning whether a list is the same as another given list.
* Adding a new integer item to the end of the list
* Removing the last given number of integers from the end of the list.
* Printing the all the integers in the list in a comma-separated format to the console.

Here is how this class is defined.

In [25]:
%%writefile ex06.cpp

#include <iostream>
#include <vector>

class MyIntegerList {
private:
     std::vector<int> list;
public:
    MyIntegerList(){}
    int count() const { return list.size(); }
    int first() const { return list[0]; }
    int last() const { return list[list.size() - 1]; }

    bool sameAs(const MyIntegerList& lst) const { return this->list == lst.list; }
    MyIntegerList& add(int e){ list.push_back(e); return *this; }
    void remove(int count) {
        if (list.size() > count) {
            for(int i = 0; i < count; i++){
                list.pop_back();
            }
        }
    }

    void print() const{
        bool first = true;
        for(int e : list) {
            if(!first) std::cout << ", ";
            else first = false;

            std::cout << e;
        }
    }
};

Writing ex06.cpp


Here is some code to test this class.

In [26]:
%%writefile -a ex06.cpp

using namespace std;

int main(){
  MyIntegerList odds;
  odds.add(1).add(3).add(5).add(7).add(9);

  cout << "     First odd element: " << odds.first() << '\n';
  cout << "      Last odd element: " << odds.last() << '\n';
  cout << "     # of odd elements: " << odds.count() << '\n';
  cout << "      All odd elements: ";
  odds.print();
  cout << endl;

  MyIntegerList evens;
  evens.add(2).add(4).add(6).add(8).add(10);
  cout << "     All even elements: ";
  evens.print();
  cout << endl;
  cout << "   Both lists the same? " << odds.sameAs(evens) << endl;

  odds.remove(2); // removes the last two elements from odds
  cout << "Remaining odd elements: ";
  odds.print();
  cout << endl;

  return 0;
}

Appending to ex06.cpp


In [27]:
!g++ -std=c++17 ex06.cpp -o ex06
!./ex06

     First odd element: 1
      Last odd element: 9
     # of odd elements: 5
      All odd elements: 1, 3, 5, 7, 9
     All even elements: 2, 4, 6, 8, 10
   Both lists the same? 0
Remaining odd elements: 1, 3, 5


We can use operator overloading to improve the usability of this class. Let's see if we can use `+` to add an element to the end of the list, `-` to remove the last given number of integers form the end of the list, `==` to tell if two lists are the same and `<<` to print the list directly. In other words we want the following code to work, which right now does not as you can see from the errors you get when run it. Uncomment the following code to see the errors.

In [28]:
%%writefile ex07.cpp

#include <iostream>
#include <vector>

class MyIntegerList {
private:
     std::vector<int> list;
public:
    MyIntegerList(){}
    int count() const { return list.size(); }
    int first() const { return list[0]; }
    int last() const { return list[list.size() - 1]; }

    bool sameAs(const MyIntegerList& lst) const { return this->list == lst.list; }
    MyIntegerList& add(int e){ list.push_back(e); return *this; }
    void remove(int count) {
        if (list.size() > count) {
            for(int i = 0; i < count; i++){
                list.pop_back();
            }
        }
    }

    void print() const{
        bool first = true;
        for(int e : list) {
            if(!first) std::cout << ", ";
            else first = false;

            std::cout << e;
        }
    }
};

using namespace std;

int main(){
  /*
  MyIntegerList a;
  // Same as: a.add(1).add(3).add(5).add(7).add(9);
  a + 1 + 3 + 5 + 7 + 9;

  MyIntegerList b;
  // Same as: b.add(2).add(4).add(6).add(8).add(10);
  b + 2 + 4 + 6 + 8 + 10;

  // Same as:
  //     cout << "  All odd elements: ";
  //     a.print();
  //     cout << endl;
  cout << "     All odd elements: " << a << endl;

  // Same as:
  //     cout << "  All even elements: ";
  //     b.print();
  //     cout << endl;
  cout << "     All even elements: " << b << endl;

  // Same as: cout << "Both lists the same? " << a.sameAs(b)
  cout << "Both lists the same? " << (a == b) << endl;

  // Same as: a.remove(2) to remove the last two integers of a
  a - 2;

  // Same as:
  //     cout << "Remaining odd elements: ";
  //     a.print();
  //     cout << endl;
  cout << "Remaining odd elements: " << a << endl;
  */

  return 0;
}

Writing ex07.cpp


In [ ]:
!g++ -std=c++17 ex07.cpp -o ex07
!./ex07

So we create a new version of this class. For the above code to work, we need to overload the following operators:
* `operator+` to suppor the `+` operator. It should behave like the `add` member function.
* `operator-` to support the `-` operator. It should behave like the `remove` member function.
* `operator==` to support the `==` operator. It should behave like the `sameAs` member function.
* `operator<<` to support the `<<` operator the prints the list directly. This must be a friend function.

Here is the new implemenation of this class. To avoid naming conflicts, we'll call it `MyIntegerList2`.

In [ ]:
%%writefile ex08.cpp

#include <iostream>
#include <vector>

class MyIntegerList {
private:
     std::vector<int> list;
public:
    MyIntegerList(){}
    int count() const { return list.size(); }
    int first() const { return list[0]; }
    int last() const { return list[list.size() - 1]; }

    bool sameAs(const MyIntegerList& lst) const { return this->list == lst.list; }
    bool operator==(const MyIntegerList& lst) const { return sameAs(lst); }

    MyIntegerList& add(int e){ list.push_back(e); return *this; }
    MyIntegerList& operator+(int e){ add(e); return *this; }

    void remove(int count) {
        if (list.size() > count) {
            for(int i = 0; i < count; i++){
                list.pop_back();
            }
        }
    }
    void operator-(int count){ remove(count); }

    void print() const{
        bool first = true;
        for(int e : list) {
            if(!first) std::cout << ", ";
            else first = false;

            std::cout << e;
        }
    }

    friend std::ostream& operator<<(std::ostream& out, const MyIntegerList& lst){
        bool first = true;
        for(int e : lst.list) {
            if(!first) std::cout << ", ";
            else first = false;

            out << e;
        }

        return out;
    }
};

Overwriting ex08.cpp


Notice how the operator functions are basically the same as their corresponding functions. Here is the code that didn't work before.

In [29]:
%%writefile -a ex08.cpp

using namespace std;

int main(){
  MyIntegerList a;
  // Same as: a.add(1).add(3).add(5).add(7).add(9);
  a + 1 + 3 + 5 + 7 + 9;

  MyIntegerList b;
  // Same as: b.add(2).add(4).add(6).add(8).add(10);
  b + 2 + 4 + 6 + 8 + 10;

  // Same as:
  //     cout << "  All odd elements: ";
  //     a.print();
  //     cout << endl;
  cout << "     All odd elements: " << a << endl;

  // Same as:
  //     cout << "  All even elements: ";
  //     b.print();
  //     cout << endl;
  cout << "     All even elements: " << b << endl;

  // Same as: cout << "Both lists the same? " << a.sameAs(b)
  cout << "Both lists the same? " << (a == b) << endl;

  // Same as: a.remove(2) to remove the last two integers of a
  a - 2;

  // Same as:
  //     cout << "Remaining odd elements: ";
  //     a.print();
  //     cout << endl;
  cout << "Remaining odd elements: " << a << endl;

  return 0;
}

Writing ex08.cpp


In [30]:
!g++ -std=c++17 ex08.cpp -o ex08
!./ex08

ex08.cpp: In function ‘int main()’:
ex08.cpp:5:3: error: ‘MyIntegerList’ was not declared in this scope
    5 |   MyIntegerList a;
      |   ^~~~~~~~~~~~~
ex08.cpp:7:3: error: ‘a’ was not declared in this scope
    7 |   a + 1 + 3 + 5 + 7 + 9;
      |   ^
ex08.cpp:9:16: error: expected ‘;’ before ‘b’
    9 |   MyIntegerList b;
      |                ^~
      |                ;
ex08.cpp:11:3: error: ‘b’ was not declared in this scope
   11 |   b + 2 + 4 + 6 + 8 + 10;
      |   ^
ex08.cpp:17:3: error: ‘cout’ was not declared in this scope
   17 |   cout << "     All odd elements: " << a << endl;
      |   ^~~~
ex08.cpp:1:1: note: ‘std::cout’ is defined in header ‘<iostream>’; did you forget to ‘#include <iostream>’?
  +++ |+#include <iostream>
    1 | 
ex08.cpp:17:45: error: ‘endl’ was not declared in this scope
   17 |   cout << "     All odd elements: " << a << endl;
      |                                             ^~~~
ex08.cpp:1:1: note: ‘std::endl’ is defined in header ‘<ostream>

As you can see this code is now working; again since we implemented the neccessary `operator` functions for the `+`, `-`, `==`, and `<<` operators.

## CODING CHALLENGE 3

Refactor (modify) the class below so as to overload the following operators:
* the `>>` operator such that `cin >> p` prompts the user to enter two integers: x and y.
* the `<<` operator such that `cout << p` prints out the point `p` in a format like this: `(10, 15)`.
* the read-only `[]` operator such that `p[0]` returns p's x and `y[1]` p's y.
* the write `[]` operator such that `p[0] = 3` sets p's x to 3 and `y[1] = 7` sets p's y to 7.


The `main` function below, create two point objects `p` and `q` and use them to call and test all the four overloaded operators.

In [33]:
%%writefile ch03.cpp

#include <iostream>
using namespace std;

class Point {
private:
    int x, y;

public:
    Point(int x, int y): x(x), y(y){}

    void print(){
        cout << "(" << this->x << ", " << this->y << ")" << endl;
    }

    int operator[](int index) const {
        switch(index){
        case 0: return this->x;
        default: return this->y;
        }
    }

    int& operator[](int index) {
        switch(index){
        case 0: return this->x;
        default: return this->y;
        }
    }

    friend ostream& operator<<(ostream& out, Point& p) {
        out << "(" << p.x << ", " << p.y << ")";
        return out;
    }

    friend istream& operator>>(istream& in, Point& p) {
        cout << "Enter an x and y coordinate (integers):" << endl;
        in >> p.x >> p.y;
        return in;
    }
};

int main()
{
  Point p(0, 0);
  Point q(0, 0);

  cin >> p;
  cin >> q;

  cout << p << endl;
  cout << q << endl;

  cout << p[0] << endl;
  cout << p[1] << endl;
  cout << q[0] << endl;
  cout << q[1] << endl;

  return 0;
}

Overwriting ch03.cpp


In [34]:
!g++ -std=c++17 ch03.cpp -o ch03
!./ch03

Enter an x and y coordinate (integers):
4 7
Enter an x and y coordinate (integers):
5 8
(4, 7)
(5, 8)
4
7
5
8
